In [ ]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

In [ ]:
import torch
print(torch.__version__)

In [ ]:
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import BaseCallback
import numpy as np

# 사용자 정의 모듈 Import
from environment_src.heater_gym_env import HeaterEnv  # (이전 단계에서 만든 환경 파일)
from environment_src.animation import TrainingAnimator # (위에서 만든 애니메이션 파일)

print("모듈 로드 완료.")

In [ ]:
import gymnasium as gym # Use gymnasium as gym (Gym is deprecated)
import pygame
from gymnasium.utils.play import play, PlayPlot

# 1. Define the callback function to compute metrics
def compute_metrics(obs_t, obs_tp1, action, rew, done, info):
    """Computes the immediate and cumulative reward."""
    # This example assumes the 'play' function handles cumulative reward internally for display
    # but the callback should return the value for each step
    return [rew, rew] # Returns immediate reward for both plots, custom logic can be used here

# 2. Instantiate PlayPlot
# horizon_timesteps specifies the time window for the live plot
# plot_names specifies the titles for each plot
plotter = PlayPlot(callback=compute_metrics, horizon_timesteps=200,
                   plot_names=["Immediate Reward", "Action Magnitude"]) # Can plot action magnitude or any other metric

# 3. Run play() with the callback
env = gym.make("CartPole-v1", render_mode="rgb_array") # Use render_mode compatible with play
# Define key to action mapping (example for CartPole)
mapping = {(pygame.K_LEFT,): 0, (pygame.K_RIGHT,): 1}
play(env, callback=plotter.callback, keys_to_action=mapping)
